In [6]:
from hamiltonians import load_Molecule

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
from typing import List 
import qutip as qt
from tqdm import tqdm
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import GroundStateEigensolver,NumPyMinimumEigensolverFactory
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureMoleculeDriver, ElectronicStructureDriverType
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper

from qiskit_nature.circuit.library import UCCSD, HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.opflow import TwoQubitReduction
from qiskit import BasicAer, Aer
from qiskit.utils import QuantumInstance
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SLSQP,SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.opflow import I, X, Z, Y
from qiskit.circuit import Parameter, ParameterVector, QuantumCircuit, QuantumRegister 

from numbers import Number

from qiskit.circuit import Gate, QuantumCircuit, QuantumRegister, ParameterExpression
from qiskit.quantum_info.operators.predicates import matrix_equal
from qiskit.quantum_info.operators.predicates import is_hermitian_matrix
from qiskit.extensions.exceptions import ExtensionError
from qiskit.circuit.exceptions import CircuitError

from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister 


from qiskit.extensions.unitary import UnitaryGate

In [36]:
import numpy as np
import pylab
import matplotlib.pyplot as plt

from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.opflow import I, X, Z

In [37]:
BeH2 = load_Molecule('BeH2', 0.5)
BeH2[0]

TaperedPauliSumOp(SparsePauliOp(['IIIIII', 'IIIIIZ', 'IIIZXX', 'IIIIYY', 'IIIIZZ', 'IIIZZI', 'IIIZII', 'IIZIII', 'ZXXIII', 'IYYIII', 'IZZIII', 'ZZIIII', 'ZIIIII', 'IIIIZI', 'IIIZZZ', 'IIIZIZ', 'IIIZYY', 'IIIIXX', 'IIZIIZ', 'IIZZXX', 'IIZIYY', 'IZXIZX', 'IIXIZX', 'IZXIIX', 'IIXIIX', 'ZXXIIZ', 'IYYIIZ', 'ZXXZXX', 'IYYZXX', 'ZXXIYY', 'IYYIYY', 'XXXXXX', 'YXYXXX', 'XXXYXY', 'YXYYXY', 'IZZIIZ', 'IZZZXX', 'IZZIYY', 'ZXZIZX', 'IXIIZX', 'ZXZIIX', 'IXIIIX', 'ZZIIIZ', 'ZZIZXX', 'ZZIIYY', 'XZIXXX', 'XIIXXX', 'XZIYXY', 'XIIYXY', 'ZIIIIZ', 'ZIIZXX', 'ZIIIYY', 'IIZIZZ', 'IZXZXZ', 'IIXZXZ', 'IZXIXI', 'IIXIXI', 'ZXXIZZ', 'IYYIZZ', 'IZZIZZ', 'ZXZZXZ', 'IXIZXZ', 'ZXZIXI', 'IXIIXI', 'XXZXXZ', 'YYIXXZ', 'XXZYYI', 'YYIYYI', 'ZZIIZZ', 'ZIIIZZ', 'IIZZZI', 'ZXXZZI', 'IYYZZI', 'XXXXZI', 'YXYXZI', 'XXXXII', 'YXYXII', 'IZZZZI', 'ZZIZZI', 'XZIXZI', 'XIIXZI', 'XZIXII', 'XIIXII', 'ZIIZZI', 'IIZZII', 'ZXXZII', 'IYYZII', 'IZZZII', 'ZZIZII', 'ZIIZII', 'IZIIII', 'ZZZIII', 'ZIZIII', 'ZYYIII', 'IXXIII'],
              co

In [38]:
def getHam(molecule, dist):
    #from qiskit_nature.units import DistanceUnit
    #from qiskit_nature.second_q.drivers import PySCFDriver
    #from qiskit_nature.second_q.mappers.qubit_converter import QubitConverter
    #from qiskit_nature.second_q.mappers import ParityMapper
    #from qiskit.opflow import TwoQubitReduction
    BeH2 = load_Molecule(molecule, dist)
    qubitham = BeH2[0]
    allcoef=[float(''.join(x.split('*')[0].split())) for x in str(qubitham).split('\n')]
    allops=[''.join(x.split('*')[1].split()) for x in str(qubitham).split('\n')]
    return allcoef,allops

In [39]:
def readOutVQE(coeffs,op_strings):
    import numpy as np
    import scipy.optimize as opt
    #from qiskit import transpile,QuantumCircuit
    #from qiskit.circuit.library import TwoLocal
    #from qiskit.providers.fake_provider import FakeAthens
    import mthree
    def opstr_to_meas_circ(op_str):
        num_qubits = len(op_str[0])
        circs = []
        for op in op_str:
            qc = QuantumCircuit(num_qubits)
            for idx, item in enumerate(op):
                if item == 'X':
                    qc.h(num_qubits-idx-1)
                elif item == 'Y':
                    qc.sdg(num_qubits-idx-1)
                    qc.h(num_qubits-idx-1)
            circs.append(qc)
        return circs
    ansatz = TwoLocal(2, rotation_blocks='ry', entanglement_blocks='cx')
    meas_circs = opstr_to_meas_circ(op_strings)
    full_circs = [ansatz.compose(meas_circs[kk]).measure_all(inplace=False) for kk in range(len(meas_circs))]
    meas_strings = [string.replace('X', 'Z').replace('Y', 'Z') for string in op_strings]
    backend = FakeAthens()
    trans_circs = transpile(full_circs, backend)  
    num_params = ansatz.num_parameters
    #params = np.random.uniform(0,5,num_params) #np.array([1.22253725, 0.39053752, 0.21462153, 5.48308027,2.06984514, 3.65227416, 4.01911194, 0.35749589])
    params = np.array([1.22253725, 0.39053752, 0.21462153, 5.48308027,
                   2.06984514, 3.65227416, 4.01911194, 0.35749589])
    params += 0.05*(np.random.random(params.shape[0])-0.5)
    mit = mthree.M3Mitigation(backend)
    mit.cals_from_system([0,1])
    def vqe_func(params, *args):
        bound_circs = [circ.bind_parameters(params) for circ in trans_circs]
        counts = backend.run(bound_circs, shots=4096).result().get_counts()
        quasis = mit.apply_correction(counts, [0,1])
        energy = np.sum(coeffs*quasis.expval(meas_strings))
        return energy

    res = opt.minimize(vqe_func, params, method = 'COBYLA')
    return res.fun

In [ ]:
coefs,ops=getHam('LiH', 0.5)
readOutVQE(coefs,ops)